In [2]:
import fasttext
import json
from faq50_adapted import FAQ, extract_word_probs
from topic_word_probs import *
from tfidf_classifier import TFIDF_Classifier
from cs_lemmatizer import *


%load_ext autoreload
%autoreload 2

In [3]:
model_path = "models/cc.cs.300.bin"
model = fasttext.load_model(model_path)

### Tests:

In [7]:
path_to_q = "upv_faq/Q78_questions.xlsx"
path_to_a = "upv_faq/Q78_answers_no_tags.xlsx"

path_to_save = "780_upv_questions/w_probs_in_q_and_a.json"
probs, n_words = count_word_probs_in_corpuses(path_to_save, path_to_questions=path_to_q, path_to_answers=path_to_a)

# with open(path_to_save, "r") as wp_file:
#     probs = json.load(wp_file)

# print(dict(sorted(probs.items(), key=lambda item: -item[1])))

In [8]:
q_xslx = "upv_faq/Q78_questions.xlsx"
a_xslx = "upv_faq/Q78_answers_no_tags.xlsx"

faq = FAQ(model, q_xslx, a_xslx, probs=probs, alpha=0.03, rm_stop_words=True, lemm=True)

In [9]:
acc, acc_sec = faq.cross_match_test()
print(f"\nQuestion Cross-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")

acc, acc_sec = faq.mean_match_test()
print(f"\nQuestion Mean-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")


Question Cross-Match Accuracy: 1st: 0.442, 2nd: 0.132

Question Mean-Match Accuracy: 1st: 0.697, 2nd: 0.131


### Mean match test without inclusion of tested question:

In [11]:
f, s, t = faq.mean_match_test_disjunctive()
print(f"\033[1m1st right: {f} \033[0m")
print(f"2nd right: {s}\n3rd right: {t}\n-> {round(f+s+t, 3)}")

1st right: 0.48 
2nd right: 0.138
3rd right: 0.072
-> 0.69


In [12]:
# pouze words from questions:
probs, n_words = count_word_probs_in_corpuses(path_to_save, path_to_questions=path_to_q, path_to_answers=None)
# histogram_of_words(probs, n_words)

In [13]:
LMTZR.remove_stop_words_from_sentence(["rešerše", "mezi", "jsem", "přihlasit", "jak", "patentem"])

['rešerše', 'přihlasit', 'patentem']

## Weighting with TF-IDF

In [3]:
path_to_q = "upv_faq/Q78_questions.xlsx"
path_to_a = "upv_faq/Q78_answers_no_tags.xlsx"

path_to_save = "780_upv_questions/w_probs_in_q_and_a.json"
# probs_old, n_words = count_word_probs_in_corpuses(path_to_save, path_to_questions=path_to_q, path_to_answers=None)

c = TFIDF_Classifier(path_to_q)
test_data = c.structure_data(test_data_percent=1) 
tfidf_matrix, feat_names = c.get_TFIDF_matrix()
probs = get_TFIDF_threshold_probabilities(tfidf_matrix, feat_names)

q_xslx = "upv_faq/Q78_questions.xlsx"
a_xslx = "upv_faq/Q78_answers.xlsx"

In [5]:
""" previous best results without tf-idf weighting:
0.471, _ : alpha=0.23

1st right: 0.481 
2nd right: 0.135
3rd right: 0.065
-> 0.681 : alpha=0.41
"""
faq = FAQ(model, q_xslx, a_xslx, 
          probs=probs, 
          alpha=0.5, 
          rm_stop_words=True, 
          lemm=True,
          tfidf_weighting=True)

acc, acc_sec = faq.cross_match_test()
print(f"\nQuestion Cross-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")

# disjunctive, but still the question is included to tfidf matrix...
f, s, t = faq.mean_match_test_disjunctive()
print(f"\n\033[1m1st right: {f} \033[0m")
print(f"2nd right: {s}\n3rd right: {t}\n-> {round(f+s+t, 3)}")


Question Cross-Match Accuracy: 1st: 0.588, 2nd: 0.115

1st right: 0.5 
2nd right: 0.145
3rd right: 0.074
-> 0.719


In [33]:
# ~ 4 minutes
acc, acc_sec = faq.cross_match_test_tfidf_disj()
print(f"\nQuestion Cross-Match Disj Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [107]:
# ~ 2 minutes
faq = FAQ(model, q_xslx, a_xslx, probs=probs, alpha=0.3, rm_stop_words=True, lemm=True,
          tfidf_weighting=True)
f, s, t = faq.mean_match_test_disjunctive(leave_one_out_also_tfidf=True)
print(f"\n\033[1m1st right: {f} \033[0m")
print(f"2nd right: {s}\n3rd right: {t}\n-> {round(f+s+t, 3)}")


1st right: 0.5 
2nd right: 0.142
3rd right: 0.071
-> 0.713


## Confused sentences:

In [8]:
# Ambiguous matches
path_to_save = "780_upv_questions/same_question_different_answers.json"
faq.get_same_question_different_answer_pairs(save_path=path_to_save)

Same question with different classes:

Q: Jaký je hlavní rozdíl mezi patentem a užitným vzorem? It's idx= 329 , true_class= 33
Q_argmax: Jaké jsou hlavní rozdíly mezi užitným vzorem a patentem? It's idx=  341 , true_class= 34

Q: Jaké jsou hlavní rozdíly mezi užitným vzorem a patentem? It's idx= 341 , true_class= 34
Q_argmax: Jaký je hlavní rozdíl mezi patentem a užitným vzorem? It's idx=  329 , true_class= 33

Q: Jaké jsou možnosti podání přihlášky průmyslového vzoru?  It's idx= 512 , true_class= 51
Q_argmax: Jaké jsou možnosti podání přihlášky průmyslového vzoru? It's idx=  490 , true_class= 49

Q: Kdo může podat přihlášku průmyslového vzoru? It's idx= 519 , true_class= 52
Q_argmax: Kdo může podat přihlášku průmyslového vzoru? It's idx=  492 , true_class= 49

Q: Jak mohu objednat rešeršní služby na Úřadu? It's idx= 550 , true_class= 55
Q_argmax: Jak mohu objednat rešeršní služby Úřadu? It's idx=  449 , true_class= 45

Q: Kdo může podat přihlášku užitného vzoru? It's idx= 592 , true_c

{'Jaký je hlavní rozdíl mezi patentem a užitným vzorem?': ['33', '34'],
 'Jaké jsou hlavní rozdíly mezi užitným vzorem a patentem?': ['34', '33'],
 'Jaké jsou možnosti podání přihlášky průmyslového vzoru? ': ['51', '49'],
 'Kdo může podat přihlášku průmyslového vzoru?': ['52', '49'],
 'Jak mohu objednat rešeršní služby na Úřadu?': ['55', '45'],
 'Kdo může podat přihlášku užitného vzoru?': ['59', '61'],
 'Jak mohu objednat rešeršní služby Úřadu?': ['65', '45'],
 'Jak dlouho trvá zpracování rešerše?': ['65', '45', '74'],
 'Co znamená kvalifikované zveřejnění technického řešení?': ['68', '58'],
 'Jakým způsobem lze podat objednávku na rešeršní službu?': ['74', '55']}

In [9]:
faq.get_most_confused_questions(cos_sim_threshold=0.85) # with mean-match

(788, 79)
(788,)
41     Na jaké stránce jsou uvedeny nejdůležitější pr...
399    Kde mohu najít právní předpisy týkající se pop...
400    Jaké jsou nejdůležitější právní předpisy týkaj...
Name: question, dtype: object


In [10]:
path_to_save = "780_upv_questions/most_missclassified_class_pairs"
faq.get_most_misclassified_class_pairs(n_of_common_misses=3, save_path=path_to_save)

(19, 45) 4
(26, 24) 3
(26, 25) 3
(30, 55) 4
(31, 69) 4
(33, 34) 4
(35, 65) 3
(36, 57) 5
(40, 4) 3
(42, 44) 3
(43, 51) 3
(43, 47) 3
(45, 55) 3
(49, 51) 5
(52, 51) 4
(53, 62) 4
(58, 68) 6
(65, 55) 3
(71, 51) 3
(77, 3) 5
['19:45', '26:24', '26:25', '30:55', '31:69', '33:34', '35:65', '36:57', '40:4', '42:44', '43:51', '43:47', '45:55', '49:51', '52:51', '53:62', '58:68', '65:55', '71:51', '77:3']


In [3]:
questions_xlsx = "780_upv_questions/expanded_data_all2.xlsx"
# questions_xlsx = "upv_faq/Q50_questions.xlsx"
c = TFIDF_Classifier(questions_xlsx)

In [4]:
test_data = c.structure_data(test_data_percent=0.1) 

c.get_TFIDF_matrix()

In [5]:
c.classify_sentence("Co by nastalo kdyby patent byl zrušen?")

73

In [6]:
c.classify_test_sentences_list(test_data)

Got right: 0.5128205128205128


In [7]:
test_data = c.structure_data(test_data_percent=1) # without removal from train data
c.get_TFIDF_matrix()
c.classify_test_sentences_list(test_data)

Got right: 0.7982233502538071


In [8]:
c.leave_one_out_test()

0it [00:00, ?it/s]

788it [01:00, 13.04it/s]


0.522

## Word embeddings based Edit Distance

In [15]:
from weed import *

rm_sw = True
lm = True

# q_xslx = "new_questions/FAQ76_questions.xlsx"
# a_xslx = "new_questions/FAQ76_answers.xlsx"
# q_xslx = "780_upv_questions/Q78_questions.xlsx"
# a_xslx = "780_upv_questions/Q78_answers_no_tags.xlsx"
q_xslx = "upv_faq/data/FAQ50_questions.xlsx"
a_xslx = "upv_faq/data/FAQ50_answers.xlsx"

c = TFIDF_Classifier(q_xslx, rm_sw, lm)
test_data = c.structure_data(test_data_percent=1) 
tfidf_matrix, feat_names = c.get_TFIDF_matrix()
probs = get_TFIDF_threshold_probabilities(tfidf_matrix, feat_names)

# Or from frequencies - change to tfidf_weighting=False
path_to_save = "w_probs_in_questions.json"
probs, n_words = count_word_probs_in_corpuses(path_to_save, path_to_questions=q_xslx, path_to_answers=None)

weed = WEED(model, q_xslx, a_xslx, probs=probs, alpha=1, 
            lemm=lm, rm_stop_words=rm_sw, sigma=0.9,
            tfidf_weighting=False)
print(weed.nearest_question_test_weed())

0.9448398576512456


: 

In [403]:
"Finding best sigma"
best_alpha_so_far = None
best_acc = 0
for sigma in np.arange(0.0, 1.0, 0.1):
    weed = WEED(model, q_xslx, a_xslx, probs=probs, alpha=0.05, 
                lemm=lm, rm_stop_words=rm_sw, sigma=sigma)
    acc = weed.nearest_question_test_weed()
    print("\n", acc)
    if acc > best_acc:
        best_acc = acc
        best_alpha_so_far = sigma

        print("S:", best_alpha_so_far)

Average number of words in a tokenized sentence: 8.803

 0.5101522842639594
S: 0.0
Average number of words in a tokenized sentence: 8.803

 0.5101522842639594
Average number of words in a tokenized sentence: 8.803

 0.5177664974619289
S: 0.2
Average number of words in a tokenized sentence: 8.803

 0.5114213197969543
Average number of words in a tokenized sentence: 8.803

 0.5215736040609137
S: 0.4
Average number of words in a tokenized sentence: 8.803

 0.5279187817258884
S: 0.5
Average number of words in a tokenized sentence: 8.803

 0.5342639593908629
S: 0.6000000000000001
Average number of words in a tokenized sentence: 8.803

 0.5317258883248731
Average number of words in a tokenized sentence: 8.803

 0.5215736040609137
Average number of words in a tokenized sentence: 8.803

 0.5177664974619289


In [284]:
"Finding best alpha"
best_alpha_so_far = None
best_acc = 0
for alpha in np.arange(0.01, 0.6, 0.022):
    weed = WEED(model, q_xslx, a_xslx, probs=probs, alpha=alpha, 
                lemm=lm, rm_stop_words=rm_sw)
    acc = weed.nearest_question_test_weed()
    if acc > best_acc:
        best_acc = acc
        best_alpha_so_far = alpha

        print(best_alpha_so_far)
        print(best_acc)

0.01
0.7764067127344522
0.032
0.7828232971372162
